In [33]:
import cirq


In [34]:
#data-preprocessing-tools
import tensorflow as tf
import numpy  as np

#quantum-machine-learning-tools
import tensorflow_quantum as tf_quantum # type: ignore


#visualization-tools 
from cirq.contrib.svg import SVGCircuit
import matplotlib.pyplot as plt

In [ ]:
#importing the fashion dataset
from tensorflow.keras.datasets  import fashion_mnist as dataset # type: ignore
from tensorflow.keras.utils import to_categorical

def load_data():
    (x_train,y_train),(x_test, y_test) = dataset.load_data()   #loading the dataset from MNIST and normalizing it with the float and diving by the 255(pix values)
    x_train = x_train.astype(np.float32) / 255.0
    x_test  = x_test.astype(np.float32) / 255.0

    x_train = np.expand_dims(x_train,axis=-1) #expanding the dimension creating a 4D tensors(batch-size, height, width, channels)
    x_test = np.expand_dims(x_test,axis=-1)

    y_train = to_categorical(y_train,num_classes=10)   # one hot encoding of the y_train and y_test using the categorical
    y_test = to_categorical(y_test, 10)

    return (x_train, y_train), (x_test, y_test)   # returnin the encoded and preprocessed dataset


#encoding the classical data into quantum data

def encode(images):
    qubits = cirq.GridQubit.rect(1,28)   #sets of images are encoded into the quantum circuits creating a grid of qubits (1 x 28)
    circuits = []
    for img in images:
        circuit = cirq.Circuit()    # for each image a quantum circuit is created and each pixel in the image is applied to a X-axis rotation corr to qubit 
        for i in range(28):   # rotation angle corr to pixel value
            for j in range(28):
                if img[i][j] > 0:
                    circuit.append(cirq.rx(img[1][j] *np.pi).on(qubits[i]))
        circuits.append(circuit)

        return tf_quantum.convert_to_tensor(circuits)  # the resulting circuits are then converted to tensors using this...
    


#building the quantum neural network model
def build_qnn(qubits):
    model_circuit = cirq.Circuit()
    neural_layer = tf_quantum.layers.PQC(model_circuit, cirq.Z(qubits[0]))  
    inputs = tf.keras.Input(shape=(), dtype=tf.dtypes.string) #input is a tensor of q. circuits and output is ameasure of the results
    outputs = neural_layer(inputs)        # creates a parameterized quantum circuit layer(taking an observable measurement) and a circuit 

    model = tf.keras.Model(inputs=inputs, outputs=outputs)  
    return model


#main function now

def main():
    (x_train, y_train), (x_test, y_test) = load_data()   #entry point of script. 
    qubits = cirq.GridQubit.rect(1,28)
    x_train_encoded = encode(x_train[:1000])  #training samples are encoded into quantum circuits using the encode function
    y_train = y_train[:1000]
 
    model = build_qnn(qubits)   # the QNN model is built using the function  and its then compiled with Adam optimizer, loss function and metric
    model.compile(optimizer=tf.keras.optimizers.Adam(learning_rate=0.01), loss = 'caterigorical_crossentropy',metrics=['accuracy'])


    model.fit(x_train_encoded, y_train,epochs=5, batch_size=32,validation_split=0.2)

    x_test_encoded = encode(x_test[:100])
    y_test = y_test[:100]

    loss,accuracy = model.evaluate(x_test_encoded, y_test)

    print(accuracy)
    


if __name__ == '__main__':   # this ensures that the main() function is only  executed when the script is run directly , and not when the script is imported as a module
    main()
